In [1]:
import pandas as pd #pandas для работы с данными
import numpy as np #numpy для математических преобразований
import matplotlib.pyplot as plt # matplotlib для визуализации
import seaborn as sns #seaborn для визуализации

In [2]:
diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


Pregnancies — количество беременностей.
Glucose — концентрация глюкозы в плазме через два часа при пероральном тесте на толерантность к глюкозе.
BloodPressure — диастолическое артериальное давление (мм рт. ст.).
SkinThickness — толщина кожной складки трицепса (мм).
Insulin — двухчасовой сывороточный инсулин (ме Ед/мл).
BMI — индекс массы тела (\(\frac{вес\ в\ кг}{(рост\ в\ м)^2}\)).
DiabetesPedigreeFunction — функция родословной диабета (чем она выше, тем выше шанс наследственной заболеваемости).
Age — возраст.
Outcome — наличие диабета (0 — нет, 1 — да).

In [3]:
#Задание 8.1
#Начнём с поиска дубликатов в данных. Найдите все повторяющиеся строки в данных и удалите их. Для поиска используйте все признаки в данных. 
#Сколько записей осталось в данных?
dupl_columns = list(diabetes.columns)


mask = diabetes.duplicated(subset=dupl_columns)
diabets_duplicates = diabetes[mask]
#print(f'Число найденных дубликатов: {diabets_duplicates.shape[0]}')
diabetes_dedupped = diabetes.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {diabetes_dedupped.shape[0]}')

Результирующее число записей: 768


In [4]:
#Задание 8.2
#Далее найдите все неинформативные признаки в данных и избавьтесь от них. В качестве порога информативности возьмите 0.95: удалите все признаки, 
# для которых 95 % значений повторяются или 95 % записей уникальны. В ответ запишите имена признаков, которые вы нашли (без кавычек).
#Примечание: дальнейшая работа производится с очищенными от дубликатов и неинформативных признаков данных.

#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in diabetes_dedupped.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes_dedupped[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes_dedupped[col].nunique() / diabetes_dedupped[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

Gender: 100.0% одинаковых значений


In [5]:
#Задание 8.2 избавление от столбца Gender
#создаем копию исходной таблицы
diabetes_df = diabetes.copy()
#Удаление столбца Gender
diabetes_df = diabetes_df.drop('Gender', axis=1)
#отображаем результирующую долю пропусков
diabetes_df.isnull().mean()

Pregnancies                 0.0
Glucose                     0.0
BloodPressure               0.0
SkinThickness               0.0
Insulin                     0.0
BMI                         0.0
DiabetesPedigreeFunction    0.0
Age                         0.0
Outcome                     0.0
dtype: float64

In [6]:
print(diabetes_df.shape)

(778, 9)


In [ ]:
#Задание 8.3
#Попробуйте найти пропуски в данных с помощью метода isnull().
#Спойлер: ничего не найдёте. А они есть! Просто они скрыты от наших глаз. 

#В таблице пропуски в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI обозначены нулём, поэтому традиционные методы поиска пропусков ничего вам не покажут. Давайте это исправим!
#Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска. Его вы можете взять из библиотеки numpy: np.nan.
#Какая доля пропусков содержится в столбце Insulin? Ответ округлите до сотых.

#Подсказка (1 из 1): Создайте функцию, которая возвращает np.nan, если её аргумент равен 0, в противном случае — возвращает аргумент. 
# Воспользуйтесь методом apply() и примените созданную функцию для каждого перечисленного столбца.

def nan_function(x):
    return np.nan if x == 0 else x
diabetes["Glucose"] = diabetes["Glucose"].apply(nan_function)
diabetes["BloodPressure"] = diabetes["BloodPressure"].apply(nan_function)
diabetes["SkinThickness"] = diabetes["SkinThickness"].apply(nan_function)
diabetes["Insulin"] = diabetes["Insulin"].apply(nan_function)
diabetes["BMI"] = diabetes["BMI"].apply(nan_function)
diabetes.isnull().mean().round(2).sort_values(ascending=False)
#0.49



In [9]:
#Задание 8.4
#Удалите из данных признаки, где число пропусков составляет более 30 %. 
# Сколько признаков осталось в ваших данных (с учетом удаленных неинформативных признаков в задании 8.2)?

#thresh = diabetes.shape[0]*0.7
#diabetes = diabetes.dropna(thresh=thresh, axis=1)
#print(diabetes.shape[1])
#8

#создаем копию исходной таблицы
drop_data = diabetes_df.copy()
#задаем минимальный порог: вычисляем 70% от числа строк
thresh = drop_data.shape[0]*0.7
#удаляем столбцы, в которых более 30% (100-70) пропусков
drop_data = drop_data.dropna(thresh=thresh, axis=1)
#удаляем записи, в которых есть хотя бы 1 пропуск
#drop_data = drop_data.dropna(how='any', axis=0)
#отображаем результирующую долю пропусков
#drop_data.isnull().mean()
print(drop_data.shape[1])


9


In [16]:
#Задание 8.5
#Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. Чему равно результирующее число записей в таблице?

m = diabetes.shape[1]
diabetes = diabetes.dropna(thresh=m-2, axis=0)
print(diabetes.shape[0])
#761



778


In [ ]:
#Задание 8.6
#В оставшихся записях замените пропуски на медиану. Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых.

null_data = diabetes.isnull().sum()
cols = null_data[null_data>0].index
for col in cols:
    diabetes[col] = diabetes[col].fillna(diabetes[col].median())
print(diabetes['SkinThickness'].mean().round(1))

#29.1 или 29,1

In [17]:
#Задание 8.7
#Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?
#Примечание: дальнейшая работа производится с очищенными от дубликатов, неинформативных признаков данных и пропусков.

def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x= data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned
outliers, _ = outliers_iqr_mod(diabetes, 'SkinThickness')
print(outliers.shape[0])
#87

1


In [ ]:
#Задание 8.8
#Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?

def outliers_z_score_mod(data, feature, left=3, right=3, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned
outliers, _ = outliers_z_score_mod(diabetes, 'SkinThickness')
print(outliers.shape[0])
#4

In [ ]:
#Задание 8.9
#На приведённой гистограмме показано распределение признака DiabetesPedigreeFunction. 
# Такой вид распределения очень похож на логнормальный, и он заставляет задуматься о логарифмировании признака. 
# Найдите сначала число выбросов в признаке DiabetesPedigreeFunction с помощью классического метода межквартильного размаха.
#Затем найдите число выбросов в этом же признаке в логарифмическом масштабе (при логарифмировании единицу прибавлять не нужно!). 
# Какова разница между двумя этими числами (вычтите из первого второе)?

def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x= data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned
outliers, _ = outliers_iqr_mod(diabetes, 'DiabetesPedigreeFunction')
outliers_log, _ = outliers_iqr_mod(diabetes, 'DiabetesPedigreeFunction', log_scale=True)
print(outliers.shape[0] - outliers_log.shape[0])
#29